In [6]:
import torch 
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from pathlib import Path
import csv
import numpy as np
import cv2

from edge_detection import edgeDetection
from interest_point_detection import display_interest_points_matrix

In [7]:
# Prepare data 

data_path = Path("./data/labels")
csv_list = list(data_path.glob("*.csv"))
file_map = dict()
i = 0

for csv_file in csv_list:

    
    
    with open(csv_file, newline='') as csvfile:

        reader = csv.reader(csvfile, delimiter=',')

        header = next(reader)

        for row in reader:
        
            if len(row) > 0:
                file_map[i] = dict()
                file_map[i]["name"] = row[0]
                
                for j in range(1, 3):
                    file_map[i][header[j]] = row[j]

            i+=1

In [8]:

img = cv2.imread(file_map[10]["name"])

_, img_filtered = edgeDetection(img, 75, 0, None, channel='r')
img_rgb = cv2.cvtColor(img_filtered, cv2.COLOR_GRAY2RGB)
                
cv2.imshow("yuh", img_filtered)
cv2.waitKey(0)
cv2.destroyAllWindows()


AttributeError: 'NoneType' object has no attribute 'suffix'

In [11]:
# Check to make sure everything is working with the data loading
for i in file_map:
    
    print(f"{i}: {file_map[i]['name']} has laser at coordinates ({file_map[i]['laser.x']}, {file_map[i]['laser.y']})")

In [30]:
# Create data matrix. Each row corresponds to one data point

training_count = int(.85*len(file_map))
testing_count = len(file_map) - training_count

X_train = np.empty((350, 12000000))
for i, file in enumerate(file_map):
    pass

[PosixPath('data/Green_Laser_Test_07_23.csv'), PosixPath('data/fred-nathanhouse-raw.csv'), PosixPath('data/label_laser_calibration.csv'), PosixPath('data/FSL-01D_Box_07_23.csv'), PosixPath('data/green_laser_07_25_23.csv'), PosixPath('data/Red_Laser_Test_07_23.csv'), PosixPath('data/FSL-01D_Fred_07_23.csv'), PosixPath('data/laser_calibration_raw_07_23.csv'), PosixPath('data/label_laser.csv'), PosixPath('data/laser_calibration_7_13_23.csv'), PosixPath('data/red_laser_07_25_23.csv'), PosixPath('data/george_measurements_07_25_23.csv')]
